In [1]:
from transformers import pipeline
import warnings
import os
warnings.simplefilter("ignore", category=FutureWarning)
os.environ["HF_HOME"] = "./HF_CACHE"

/Users/kutay-gpu/.pyenv/versions/huggingface-client/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
pipe_args = {
    "task": "object-detection",
    "model": "hustvl/yolos-tiny"
}

inference_config = {
    "input": "https://lh7-us.googleusercontent.com/GNVsVvVmCDLtFtqburTWaTQOjXn3N0wLBKXt6BhXO2GG037S9o7xZ7HyPjFcJoWgds1PFKvbHgCCEWTqiB-MR0VFIo7UC7jVEX6t10LD6zPg61YoEAR-NGbA2RJj7dVdYwMH2nsOJY648uXmMlpNiZw",
    "args": {
        "threshold": 0.5  # minimum confidence score
    }
}

In [19]:
# Load the pipeline
pipe = pipeline(**pipe_args)

# Run inference
output = pipe(
    inference_config["input"],
    **inference_config["args"]
)
# Print the output
print(output)

Device set to use mps:0


[{'score': 0.5644752383232117, 'label': 'truck', 'box': {'xmin': 702, 'ymin': 63, 'xmax': 1007, 'ymax': 198}}, {'score': 0.9994232654571533, 'label': 'dog', 'box': {'xmin': 129, 'ymin': 288, 'xmax': 444, 'ymax': 840}}, {'score': 0.9868428111076355, 'label': 'bicycle', 'box': {'xmin': 110, 'ymin': 152, 'xmax': 882, 'ymax': 623}}, {'score': 0.959194004535675, 'label': 'car', 'box': {'xmin': 697, 'ymin': 63, 'xmax': 1007, 'ymax': 210}}]


In [23]:
from typing import List, Dict, Any
from pydantic import BaseModel

class DetectionResponse(BaseModel):
    results: List[Dict[str, Any]]
    
    def display_results(self):
        """Display detection results in a nice format"""
        print("🔍 Object Detection Results")
        print("=" * 50)
        
        # Sort by confidence score (highest first)
        sorted_results = sorted(self.results, key=lambda x: x['score'], reverse=True)
        
        for i, result in enumerate(sorted_results, 1):
            confidence_percent = result['score'] * 100
            
            # Color coding based on confidence
            if confidence_percent >= 90:
                confidence_emoji = "🟢"
            elif confidence_percent >= 70:
                confidence_emoji = "🟡"
            else:
                confidence_emoji = "🔴"
                
            print(f"\n{i}. {confidence_emoji} {result['label'].upper()}")
            print(f"   Confidence: {confidence_percent:.1f}%")
            box = result['box']
            print(f"   Bounding Box: ({box['xmin']}, {box['ymin']}) → ({box['xmax']}, {box['ymax']})")
            
            # Calculate box dimensions
            width = box['xmax'] - box['xmin']
            height = box['ymax'] - box['ymin']
            print(f"   Dimensions: {width}x{height} pixels")
    

In [ ]:
# Direkt output'u kullan, dönüştürmeye gerek yok!
detection_response = DetectionResponse(results=output)
detection_response.display_results()

🔍 Object Detection Results

1. 🟢 DOG
   Confidence: 99.9%
   Bounding Box: (129, 288) → (444, 840)
   Dimensions: 315x552 pixels

2. 🟢 BICYCLE
   Confidence: 98.7%
   Bounding Box: (110, 152) → (882, 623)
   Dimensions: 772x471 pixels

3. 🟢 CAR
   Confidence: 95.9%
   Bounding Box: (697, 63) → (1007, 210)
   Dimensions: 310x147 pixels

4. 🔴 TRUCK
   Confidence: 56.4%
   Bounding Box: (702, 63) → (1007, 198)
   Dimensions: 305x135 pixels
